# GI Prediction Category Testing

This notebook tests our GI prediction system across different recipe categories:
1. Simple recipes
2. High GI recipes
3. Low GI recipes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from agents.gi_agent import GIAnalysisAgent
from agents.gi_grader_agent import GIGraderAgent
from test_data.simple.simple_recipes import SIMPLE_RECIPES
from test_data.high_gi.high_gi_recipes import HIGH_GI_RECIPES
from test_data.low_gi.low_gi_recipes import LOW_GI_RECIPES
import json
from typing import List, Dict, Any
from datetime import datetime
import os

## Setup Results Directory

In [ ]:
# Create timestamp for this test run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = f'test_results/{timestamp}'
plots_dir = f'{results_dir}/plots'
data_dir = f'{results_dir}/data'

# Create directories if they don't exist
os.makedirs(plots_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

print(f"Results will be saved in: {results_dir}")

## Initialize Agents

In [ ]:
# Initialize agents
gi_agent = GIAnalysisAgent()
grader_agent = GIGraderAgent()

## Test Function

In [ ]:
def test_category(recipes: List[Dict[str, Any]], category_name: str) -> pd.DataFrame:
    """Test a category of recipes and return results"""
    results = []
    
    for recipe in recipes:
        # Get our prediction
        our_result = gi_agent.process([recipe])
        if 'error' in our_result:
            print(f"Error processing {recipe['title']}: {our_result['error']}")
            continue
            
        our_prediction = our_result['glycemic_load']
        
        # Get expert grading
        grade_result = grader_agent.grade_prediction(recipe, our_prediction)
        
        # Store results
        results.append({
            'recipe': recipe['title'],
            'category': category_name,
            'our_prediction': our_prediction,
            'expert_assessment': grade_result['assessed_gi'],
            'prediction_difference': grade_result['prediction_difference']
        })
    
    return pd.DataFrame(results)

## Run Tests

In [ ]:
# Test each category
simple_results = test_category(SIMPLE_RECIPES, "Simple")
high_gi_results = test_category(HIGH_GI_RECIPES, "High GI")
low_gi_results = test_category(LOW_GI_RECIPES, "Low GI")

# Combine results
all_results = pd.concat([simple_results, high_gi_results, low_gi_results])
all_results

## Visualize Results

In [ ]:
plt.figure(figsize=(15, 10))

# Box plot of prediction differences by category
plt.subplot(2, 2, 1)
sns.boxplot(x='category', y='prediction_difference', data=all_results)
plt.title('Prediction Differences by Category')
plt.xticks(rotation=45)
plt.ylabel('Absolute Difference')

# Scatter plot of our predictions vs expert assessments
plt.subplot(2, 2, 2)
sns.scatterplot(data=all_results, x='our_prediction', y='expert_assessment', hue='category')
plt.title('Our Predictions vs Expert Assessments')
plt.xlabel('Our Prediction')
plt.ylabel('Expert Assessment')

# Add diagonal line for perfect predictions
max_val = max(all_results['our_prediction'].max(), all_results['expert_assessment'].max())
plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)

# Box plot of percentage differences by category
plt.subplot(2, 2, 3)
sns.boxplot(x='category', y='percentage_difference', data=all_results)
plt.title('Percentage Differences by Category')
plt.xticks(rotation=45)
plt.ylabel('Percentage Difference (%)')

# Distribution of prediction differences
plt.subplot(2, 2, 4)
sns.histplot(data=all_results, x='prediction_difference', hue='category', multiple='stack')
plt.title('Distribution of Prediction Differences')
plt.xlabel('Prediction Difference')
plt.ylabel('Count')

plt.tight_layout()
plt.savefig(f'{plots_dir}/gi_analysis.png')
plt.show()

## Analysis

In [ ]:
# Calculate additional metrics
all_results['percentage_difference'] = (all_results['prediction_difference'] / all_results['expert_assessment']) * 100
rmse = np.sqrt(np.mean(all_results['prediction_difference'] ** 2))

print("\nOverall Statistics:")
print("=================")
print(f"Total recipes tested: {len(all_results)}")
print(f"Mean prediction difference: {all_results['prediction_difference'].mean():.2f}")
print(f"Mean percentage difference: {all_results['percentage_difference'].mean():.2f}%")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
print(f"Standard deviation: {all_results['prediction_difference'].std():.2f}")
print(f"Max difference: {all_results['prediction_difference'].max():.2f}")
print(f"Min difference: {all_results['prediction_difference'].min():.2f}")

print("\nCategory-wise Statistics:")
print("======================")
for category in ['Simple', 'High GI', 'Low GI']:
    cat_results = all_results[all_results['category'] == category]
    cat_rmse = np.sqrt(np.mean(cat_results['prediction_difference'] ** 2))
    print(f"\n{category}:")
    print(f"Number of recipes: {len(cat_results)}")
    print(f"Mean difference: {cat_results['prediction_difference'].mean():.2f}")
    print(f"Mean percentage difference: {cat_results['percentage_difference'].mean():.2f}%")
    print(f"RMSE: {cat_rmse:.2f}")
    print(f"Standard deviation: {cat_results['prediction_difference'].std():.2f}")

## Export Results

In [ ]:
# Save detailed results to CSV
all_results.to_csv(f'{data_dir}/gi_test_results.csv', index=False)

# Save summary statistics
summary_stats = {
    'overall': {
        'total_recipes': len(all_results),
        'mean_difference': all_results['prediction_difference'].mean(),
        'mean_percentage_difference': all_results['percentage_difference'].mean(),
        'rmse': rmse,
        'std_difference': all_results['prediction_difference'].std(),
        'max_difference': all_results['prediction_difference'].max(),
        'min_difference': all_results['prediction_difference'].min()
    },
    'by_category': {
        category: {
            'count': len(all_results[all_results['category'] == category]),
            'mean_difference': all_results[all_results['category'] == category]['prediction_difference'].mean(),
            'mean_percentage_difference': all_results[all_results['category'] == category]['percentage_difference'].mean(),
            'rmse': np.sqrt(np.mean(all_results[all_results['category'] == category]['prediction_difference'] ** 2)),
            'std_difference': all_results[all_results['category'] == category]['prediction_difference'].std()
        }
        for category in ['Simple', 'High GI', 'Low GI']
    }
}

with open(f'{data_dir}/summary_stats.json', 'w') as f:
    json.dump(summary_stats, f, indent=4)

print(f"\nResults have been saved to {results_dir}")